In [2]:
import os
import sys

import pandas as pd
import numpy as np
import typing

sys.path.append(os.path.abspath("../common"))  # add path to common functions
from evaluate import evaluate_predictions
from preprocess import getdfs, get_char_columns
from preprocess import one_hot_encode
from models import predict_logistic_regression
from models import grid_search_
from models import predict_xgboost

In [5]:
emo_df, emo_val_df = getdfs('emoticon')

In [6]:
emo_df = get_char_columns(emo_df)
emo_df.head()


,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,label
0,😛,🛐,😻,😑,😣,🙠,🙯,🚼,😒,🙼,😑,🙯,😣,0
1,🛐,😑,😪,😛,🚼,🙯,😣,🚅,😑,🙯,😹,😣,🙼,0
2,😛,🙯,😑,🚡,😣,🚼,🛐,🙲,😣,🙯,🛑,😑,🙼,0
3,😛,🚼,🛐,🙐,😣,🙯,😑,🙪,😑,🙼,🛆,😣,🙯,1
4,🛐,🚟,🚼,😛,🙋,😑,😣,🙯,😹,🙯,😑,😣,🙼,1


In [7]:

repeat_emos = ['🙼', '🛐', '🙯', '😛', '😣', '😑', '🚼']

repeat_emo_code = {
     '🙼' : '284',
     '🛐' : '464', 
     '🙯' : '262',
     '😛' : '15436', 
     '😣' : '614',
     '😑' : '1596', 
     '🚼' : '422'
}

In [8]:
train_df, valid_df = getdfs('text_seq')

In [9]:
def remove_substrings(input_string, substrings):
    """
    Removes all occurrences of substrings from the input string.

    Parameters:
    input_string (str): The string to remove substrings from.
    substrings (list): List of substrings to remove from the input string.

    Returns:
    str: The input string with substrings removed.
    """
    for substring in substrings:
        input_string = input_string.replace(substring, "")
    return input_string


In [10]:
def process_strings ( strs : typing.List[str] )-> typing.List[str]:
    strs = [x.lstrip('0') for x in strs]

    # Example usage
    substrings = repeat_emo_code.values()

    # Remove the substrings
    strs = [remove_substrings(input_string, substrings) for input_string in strs]

    padded_strs = []

    for s in strs:
        if len(s) < 15:
            s = s + '0'*(15-len(s))
        padded_strs.append(s)

    return padded_strs

In [11]:
train_df['input_str'] = process_strings(train_df['input_str'])
valid_df['input_str'] = process_strings(valid_df['input_str'])